# Different Configuration using Homeostasis with Kmeans

In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import numpy as np
import matplotlib.pyplot as plt


from HOTS.Monitor import GenerateAM, DisplayAM
from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import DisplayImage, DisplaySurface3D, DisplaySurface2D, DisplayConvergence
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.KmeansHomeoCluster import KmeansHomeo

# 1. Defining Common parameter

In [15]:
tau = 0.02
R = 2
nb_cluster = [4,8,16]
eta_list = [1e-5,1e-5] 
eta_homeo_list = [1e-3,1e-3]
C_list = [6,18] 
ImageSize = (32,32)
DataPath = '../Data/ExtractedStabilized.mat'
LabelPath = '../Data/alphabet_label.pkl'

# 2. StepbyStep Training

In [ ]:

image_list = [1, 32, 19, 22, 29]
event0 = Event(ImageSize=(32,32),OutOnePolarity=False)

event0.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=image_list)

## Layer1

SpTe_Layer1= STS(tau=tau, R=R,verbose=0)
Surface_Layer1 = SpTe_Layer1.create(event = event0,kernel='linear')
event_filtered1, filt = SpTe_Layer1.FilterRecent(event = event0, threshold=2)
ClusterLayer1 = KmeansHomeo(nb_cluster = nb_cluster[0], eta=eta_list[0], eta_homeo=eta_homeo_list[0],
                            C=C_list[0], to_record=True, verbose=0)
Prototype1 = ClusterLayer1.fit(SpTe_Layer1,NbCycle=2,record_num_batches=10000)
event2,_ = ClusterLayer1.predict(Surface=SpTe_Layer1.Surface,event = event_filtered1)
## Layer2
SpTe_Layer2= STS(tau=10*tau, R=2*R,verbose=0)
Surface_Layer2 = SpTe_Layer2.create(event = event2,kernel='linear')
ClusterLayer2 = KmeansHomeo(nb_cluster = nb_cluster[1], eta=eta_list[1], eta_homeo=eta_homeo_list[1],
                            C=C_list[1],to_record=True, verbose=0)
Prototype2 = ClusterLayer2.fit(SpTe_Layer2,NbCycle=2,record_num_batches=10000)
event3,_ = ClusterLayer2.predict(Surface=SpTe_Layer2.Surface,event = event2)

## Layer3
#SpTe_Layer3= STS(tau=10*10*tau, R=2*2*R,verbose=0)
#Surface_Layer3 = SpTe_Layer3.create(event = event3,kernel='linear')
#ClusterLayer3 = KmeansHomeo(nb_cluster = nb_cluster[2], to_record=True, verbose=1)
#Prototype3 = ClusterLayer3.fit(SpTe_Layer3,NbCycle=0.2)
#event4,_ = ClusterLayer3.predict(Surface=SpTe_Layer3.Surface,event = event3)


In [ ]:
DisplaySurface2D(ClusterLayer1.prototype,nb_polarities=2)
DisplaySurface2D(ClusterLayer2.prototype, nb_polarities=4)
DisplayConvergence([ClusterLayer1,ClusterLayer2],to_display=['error','histo'])

## Exp1 :  Homeo Kmeans + Linear Kernel + NoiseFilter(=2) +  One Polarity in Input 

In [ ]:
save_exp = False
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(NbTrainingData=36, NbTestingData=40,
                                                        Path='../Data/ExtractedStabilized.mat',
                                                        LabelPath='../Data/alphabet_label.pkl',
                                                        ClusteringData=[1, 32, 19, 22, 29],
                                                        OutOnePolarity = True, verbose=0)
## Defining the network
L1 = ClusteringLayer(tau=tau, R=R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
L2 = ClusteringLayer(tau=10*tau, R=2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
L3 = ClusteringLayer(tau=10*10*tau, R=2*2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
Net = Network([L1,L2,L3])
#Net = Network([L1])#,L2,L3])

if save_exp == True :
    #### Training The Cluster on 5 images
    ClusterLayer, event_output = Net.TrainCluster(event = event_cl,nb_cluster=nb_cluster, KN=KN,
                                                 record_each=1000, NbCycle=1.5)

    ## Generating histogram for training samples
    event0_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

    ## Generating histogram for the test samples
    event0_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)

    Classif0 = Classifier(event0_o_tr,event0_o_te, TrainingLabel=label_tr, GroundTruth=label_te)

    eucli,norma,battach,accu = Classif0.HistogramDistance()
    
    SaveObject([ClusterLayer,Classif0],'../Records/EXP_03_HomeoKmeans/Exp1_2017_10_18.pkl')
    
else : 
    ClusterLayer,Classif0 = LoadObject('../Records/EXP_03_HomeoKmeans/Exp1_2017_10_18.pkl')
    eucli,norma,battach,accu = Classif0.HistogramDistance()

In [ ]:
DisplaySurface2D(ClusterLayer[0].prototype,nb_polarities=1, scale=2)
DisplaySurface2D(ClusterLayer[1].prototype, nb_polarities=4, scale=2)
DisplaySurface2D(ClusterLayer[2].prototype, nb_polarities=8, scale=2)
event_to_show = Event(ImageSize=(32,32), ListPolarities=[-1,1],OutOnePolarity=True)
event_to_show.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=22)
Net.RunNetwork(event_to_show, ClusterList=ClusterLayer)
mapL1 = GenerateAM(Net.Layers[0].output,ClusterLayer[0],nb_image=0)
DisplayAM(mapL1,scale=2)
mapL2 = GenerateAM(Net.Layers[1].output,ClusterLayer[1],nb_image=0)
DisplayAM(mapL2,scale=2)
mapL3 = GenerateAM(Net.Layers[2].output,ClusterLayer[2],nb_image=0)
DisplayAM(mapL3,scale=2)

DisplayConvergence(ClusterLayer)

In [ ]:
ClusterLayer[0].record['error'].values

## Exp2 :  Homeo Kmeans + Linear Kernel + NoiseFilter(=2) +  2 Polarity in Input 

In [ ]:
save_exp = False
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(NbTrainingData=36, NbTestingData=40,
                                                        Path='../Data/ExtractedStabilized.mat',
                                                        LabelPath='../Data/alphabet_label.pkl',
                                                        ClusteringData=[1, 32, 19, 22, 29],
                                                        OutOnePolarity = False, verbose=0)
## Defining the network
L1 = ClusteringLayer(tau=tau, R=R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
L2 = ClusteringLayer(tau=10*tau, R=2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
L3 = ClusteringLayer(tau=10*10*tau, R=2*2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='linear')
Net = Network([L1,L2,L3])
#Net = Network([L1])#,L2,L3])

if save_exp == True :
    #### Training The Cluster on 5 images
    ClusterLayer, event_output = Net.TrainCluster(event = event_cl,nb_cluster=nb_cluster, KN=KN,
                                                 record_each=1000, NbCycle=1.5)

    ## Generating histogram for training samples
    event0_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

    ## Generating histogram for the test samples
    event0_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)

    Classif0 = Classifier(event0_o_tr,event0_o_te, TrainingLabel=label_tr, GroundTruth=label_te)

    eucli,norma,battach,accu = Classif0.HistogramDistance()
    
    SaveObject([ClusterLayer,Classif0],'../Records/EXP_03_HomeoKmeans/Exp2_2017_10_18.pkl')
    
else : 
    ClusterLayer,Classif0 = LoadObject('../Records/EXP_03_HomeoKmeans/Exp2_2017_10_18.pkl')
    eucli,norma,battach,accu = Classif0.HistogramDistance()

In [ ]:
DisplaySurface2D(ClusterLayer[0].prototype,nb_polarities=2, scale=2)
DisplaySurface2D(ClusterLayer[1].prototype, nb_polarities=4, scale=2)
DisplaySurface2D(ClusterLayer[2].prototype, nb_polarities=8, scale=2)
event_to_show = Event(ImageSize=(32,32), ListPolarities=[-1,1],OutOnePolarity=False)
event_to_show.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=22)
Net.RunNetwork(event_to_show, ClusterList=ClusterLayer)
mapL1 = GenerateAM(Net.Layers[0].output,ClusterLayer[0],nb_image=0)
DisplayAM(mapL1,scale=2)
mapL2 = GenerateAM(Net.Layers[1].output,ClusterLayer[1],nb_image=0)
DisplayAM(mapL2,scale=2)
mapL3 = GenerateAM(Net.Layers[2].output,ClusterLayer[2],nb_image=0)
DisplayAM(mapL3,scale=2)

DisplayConvergence(ClusterLayer)


## Exp3 :  Homeo Kmeans + Exp Kernel + NoiseFilter(=2) +  1 Polarity in Input 

## Exp4 :  Homeo Kmeans + Exp Kernel + NoiseFilter(=2) +  2 Polarity in Input 

In [ ]:
save_exp = False
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(NbTrainingData=36, NbTestingData=40,
                                                        Path='../Data/ExtractedStabilized.mat',
                                                        LabelPath='../Data/alphabet_label.pkl',
                                                        ClusteringData=[1, 32, 19, 22, 29],
                                                        OutOnePolarity = False, verbose=0)
## Defining the network
L1 = ClusteringLayer(tau=tau, R=R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='exponential')
L2 = ClusteringLayer(tau=10*tau, R=2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='exponential')
L3 = ClusteringLayer(tau=10*10*tau, R=2*2*R, verbose=0, ThrFilter=2, LearningAlgo='homeo',kernel='exponential')
Net = Network([L1,L2,L3])
#Net = Network([L1])#,L2,L3])

if save_exp == True :
    #### Training The Cluster on 5 images
    ClusterLayer, event_output = Net.TrainCluster(event = event_cl,nb_cluster=nb_cluster, KN=KN,
                                                 record_each=1000, NbCycle=1.5)

    ## Generating histogram for training samples
    event0_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

    ## Generating histogram for the test samples
    event0_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)

    Classif0 = Classifier(event0_o_tr,event0_o_te, TrainingLabel=label_tr, GroundTruth=label_te)

    eucli,norma,battach,accu = Classif0.HistogramDistance()
    
    SaveObject([ClusterLayer,Classif0],'../Records/EXP_03_HomeoKmeans/Exp4_2017_10_18.pkl')
    
else : 
    ClusterLayer,Classif0 = LoadObject('../Records/EXP_03_HomeoKmeans/Exp4_2017_10_18.pkl')
    eucli,norma,battach,accu = Classif0.HistogramDistance()

In [ ]:
DisplaySurface2D(ClusterLayer[0].prototype,nb_polarities=2)
DisplaySurface2D(ClusterLayer[1].prototype, nb_polarities=4)
DisplaySurface2D(ClusterLayer[2].prototype, nb_polarities=8)
event_to_show = Event(ImageSize=(32,32), ListPolarities=[-1,1],OutOnePolarity=False)
event_to_show.LoadFromMat('../Data/ExtractedStabilized.mat',image_number=22)
Net.RunNetwork(event_to_show, ClusterList=ClusterLayer)
mapL1 = GenerateAM(Net.Layers[0].output,ClusterLayer[0],nb_image=0)
DisplayAM(mapL1,scale=2)
mapL2 = GenerateAM(Net.Layers[1].output,ClusterLayer[1],nb_image=0)
DisplayAM(mapL2,scale=2)
mapL3 = GenerateAM(Net.Layers[2].output,ClusterLayer[2],nb_image=0)
DisplayAM(mapL3,scale=2)

DisplayConvergence(ClusterLayer)

In [ ]:
Couche 1 : 2 filters 2 pol
Couche 2 : 8 filters (16) 2 pola
Couche 3 : 8 filters